In [2]:
import tensorflow as tf
from tensorflow.keras.applications import VGG19, DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = 'C:/Users/HP/Desktop/train_test_split_ext/sent3/images_hibiscus_ext/train'
test_dir = 'C:/Users/HP/Desktop/train_test_split_ext/sent3/images_hibiscus_ext/test'
validation_dir = 'C:/Users/HP/Desktop/train_test_split_ext/sent3/images_hibiscus_ext/val'

# Data augmentation for training data
train_datagen = ImageDataGenerator(rescale=1./255)

# Only rescaling for validation and test data
val_test_datagen = ImageDataGenerator(rescale=1./255)

# Load data from directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

validation_generator = val_test_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

# Load base models without the top layers
input_tensor = Input(shape=(224, 224, 3))
vgg19_base = VGG19(weights='imagenet', include_top=False, input_tensor=input_tensor)
densenet_base = DenseNet121(weights='imagenet', include_top=False, input_tensor=input_tensor)

# Freeze base model layers
for layer in vgg19_base.layers:
    layer.trainable = False
for layer in densenet_base.layers:
    layer.trainable = False

# Extract features
vgg19_output = GlobalAveragePooling2D()(vgg19_base.output)
densenet_output = GlobalAveragePooling2D()(densenet_base.output)

# Concatenate features
merged = Concatenate()([vgg19_output, densenet_output])
merged = Dense(1024, activation='relu')(merged)
predictions = Dense(1, activation='sigmoid')(merged)

# Define final model
model = Model(inputs=input_tensor, outputs=predictions)
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=15
)

test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')


Found 2238 images belonging to 2 classes.
Found 482 images belonging to 2 classes.
Found 480 images belonging to 2 classes.


C:\Users\HP\Documents\anaconda2\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
69/69 ━━━━━━━━━━━━━━━━━━━━ 656s 9s/step - accuracy: 0.8655 - loss: 0.3512 - val_accuracy: 0.9875 - val_loss: 0.0706
Epoch 2/15
 1/69 ━━━━━━━━━━━━━━━━━━━━ 8:02 7s/step - accuracy: 1.0000 - loss: 0.0465

C:\Users\HP\Documents\anaconda2\Lib\site-packages\keras\src\trainers\epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


69/69 ━━━━━━━━━━━━━━━━━━━━ 118s 2s/step - accuracy: 1.0000 - loss: 0.0465 - val_accuracy: 0.9917 - val_loss: 0.0693
Epoch 3/15
69/69 ━━━━━━━━━━━━━━━━━━━━ 608s 9s/step - accuracy: 0.9941 - loss: 0.0570 - val_accuracy: 0.9979 - val_loss: 0.0354
Epoch 4/15
69/69 ━━━━━━━━━━━━━━━━━━━━ 119s 2s/step - accuracy: 1.0000 - loss: 0.0330 - val_accuracy: 0.9937 - val_loss: 0.0350
Epoch 5/15
69/69 ━━━━━━━━━━━━━━━━━━━━ 617s 9s/step - accuracy: 0.9956 - loss: 0.0291 - val_accuracy: 0.9979 - val_loss: 0.0221
Epoch 6/15
69/69 ━━━━━━━━━━━━━━━━━━━━ 115s 2s/step - accuracy: 1.0000 - loss: 0.0159 - val_accuracy: 0.9979 - val_loss: 0.0221
Epoch 7/15
69/69 ━━━━━━━━━━━━━━━━━━━━ 620s 9s/step - accuracy: 0.9974 - loss: 0.0189 - val_accuracy: 0.9979 - val_loss: 0.0177
Epoch 8/15
69/69 ━━━━━━━━━━━━━━━━━━━━ 145s 2s/step - accuracy: 1.0000 - loss: 0.0072 - val_accuracy: 0.9979 - val_loss: 0.0182
Epoch 9/15
69/69 ━━━━━━━━━━━━━━━━━━━━ 627s 9s/step - accuracy: 0.9998 - loss: 0.0139 - val_accuracy: 0.9979 - val_loss: 0.

In [3]:
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')
print(f'Validation Loss: {val_loss}')
print(f'Validation Accuracy: {val_accuracy}')


16/16 ━━━━━━━━━━━━━━━━━━━━ 110s 7s/step - accuracy: 0.9937 - loss: 0.0126
Test Loss: 0.006247567478567362
Test Accuracy: 1.0
Validation Loss: 0.007028054445981979
Validation Accuracy: 0.9979252815246582
